In [ ]:
!pip install azureml.automl.runtime

In [1]:
import pickle
import pandas as pd
import numpy as np

test_data = pd.read_csv("../datasets/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")
print("test_data: "+ str(test_data.shape))
X_test=test_data.iloc[1:10,1:785]
print("X_test: " +str(X_test.shape))

test_data: (7172, 785)
X_test: (9, 784)


In [2]:
test_data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7167,1,135,119,108,102,105,99,61,103,121,...,108,112,116,114,118,180,184,176,167,163
7168,12,157,159,161,164,166,166,171,174,175,...,213,213,213,214,213,211,210,210,209,208
7169,2,190,191,190,191,190,190,192,192,191,...,216,215,213,214,214,213,210,211,209,208
7170,4,201,205,208,209,214,216,218,223,226,...,112,169,255,255,237,113,91,67,70,63


In [3]:
filename = '../models/AutoML05929722a48/model.pkl'
skl_model =pickle.load(open(filename, 'rb'))

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
result =skl_model.predict(X_test)
print(result)

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType,Int64TensorType

initial_type = [('pixels', Int64TensorType([None, 784]))]
onx = convert_sklearn(skl_model, initial_types=initial_type)

In [ ]:
import onnxruntime as rt
x_test_array = np.array(X_test, dtype=np.int64)
x_test_array


In [ ]:
sess = rt.InferenceSession(onx.SerializeToString())

In [ ]:
input_name= sess.get_inputs()[0].name

In [ ]:
label_name = sess.get_outputs()[0].name

In [ ]:
pred_onx = sess.run([label_name], {input_name: x_test_array})[0]
pred_onx

In [ ]:
## Model Performance

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis =1)
y_test = np.argmax(y_test, axis =1)

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
print("Accuracy :" + str(round(accuracy_score(y_test, y_pred),4)))
p, r, f, s = precision_recall_fscore_support(y_test, y_pred, average='macro')
print("precision: " + round(p,4))
print("Recall: "+round(r,4))
print("F1: "+round(f,4))